# Toronto

##  Background
Toronto, a city in southeast Canada, capital of the province of Ontario, located in the southern part of the province, on the shores of Lake Ontario and at the mouth of the Humber and Don rivers. The city is the economic, commercial, industrial and cultural centre of the country and the largest metropolitan area in Canada. Toronto is the largest city in Ontario and the second most populous in Canada. Today, it is a cosmopolitan city that is home to many different cultures and ethnicities. Toronto is the most popular destination for newcomers to Canada, with first-generation immigrants making up 47% of the city’s population. The city’s popularity is understandable as Toronto is Canada’s economic hub, boasting North America’s fastest growing tech-jobs market.

In [ ]:
## Problem
How to plan to start a business related to immgration services in this area? Which location is most popular for immf

## Interest 
This study is valuable for investors and government.

## step1: prepare data

In [14]:
!conda install -c conda-forge geocoder --yes
print("Installation Done!")
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

Solving environment: \ ^C
failed

CondaError: KeyboardInterrupt

Installation Done!


In [2]:
web_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
web_text= requests.get(web_link).text
Canada_data = BeautifulSoup(web_text, 'lxml')

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())
print('soup ready')

soup ready


In [4]:
data = []
columns = []
table = soup.find(class_='wikitable')

for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
  
    if (index == 0):
        columns = section
    else:
        data.append(section)

In [5]:
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [7]:
canada_df["Neighbourhood"] = canada_df.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))
canada_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"


In [8]:
# Remove duplicate entries
canada_df = canada_df.drop_duplicates()
canada_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park


In [9]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
canada_df['Neighbourhood'].replace("Not assigned", canada_df["Borough"],inplace=True)
canada_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park


In [10]:
canada_df.shape

(103, 3)

## Step2:Geo

In [23]:
 pip install geocoder


The following command must be run outside of the IPython shell:

    $ pip install geocoder

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [24]:
import geocoder # import geocoder

In [25]:
def get_geocoder(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

In [27]:
canada_df['Latitude'], canada_df['Longitude'] = zip(*canada_df['Postcode'].apply(get_geocoder))
canada_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.752420,-79.329242
3,M4A,North York,Victoria Village,43.730600,-79.313265
4,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
5,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286
7,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715


## Step3

In [29]:
toronto_df = canada_df[canada_df['Borough'].str.contains('Toronto')]
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
7,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715
16,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180
33,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481
46,M4E,East Toronto,The Beaches,43.676531,-79.295425


In [30]:
# create an initial map of Toronto
address = "Toronto, ON"
geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude

map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=12)


# add markers of neighborhoods onto the initial map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc3131',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto) 
    
map_toronto